In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microbes-dataset/microbes.csv


In [2]:
!pip install flaml

from sklearn.model_selection import train_test_split
from flaml import AutoML
from sklearn.decomposition import PCA
import pandas_profiling
import seaborn as sns
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 KB 369.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.6.1
    Uninstalling xgboost-1.6.1:
      Successfully uninstalled xgboost-1.6.1


In [3]:
MData = pd.read_csv('/kaggle/input/microbes-dataset/microbes.csv', index_col=0)
MData

,Solidity,Eccentricity,EquivDiameter,Extrema,FilledArea,Extent,Orientation,EulerNumber,BoundingBox1,BoundingBox2,...,ConvexHull4,MajorAxisLength,MinorAxisLength,Perimeter,ConvexArea,Centroid1,Centroid2,Area,raddi,microorganisms
0,10.70,15.8,5.43,3.75,0.7850,8.14,2.15,22.3,2.97,10.90,...,2.97,1.340,1.610,0.683,0.1950,3.63,12.10,1.310,7.99,Spirogyra
1,5.60,18.3,4.14,6.16,0.3640,3.51,18.60,22.5,5.41,19.20,...,5.47,1.520,1.520,1.010,0.2150,6.01,20.60,0.765,7.99,Spirogyra
2,8.32,19.8,4.63,6.66,0.4150,5.85,21.00,22.4,5.96,10.20,...,5.96,1.630,1.380,1.110,0.1820,6.55,11.50,0.953,7.99,Spirogyra
3,10.10,17.9,7.29,11.10,1.4700,6.30,9.94,21.9,8.81,10.70,...,8.88,2.040,2.120,0.715,0.3710,10.30,12.00,2.340,7.99,Spirogyra
4,6.27,20.2,20.10,10.70,14.7000,3.97,2.58,11.9,10.20,1.22,...,10.20,7.780,6.210,6.800,4.4400,14.00,9.55,17.600,7.99,Spirogyra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30522,3.01,22.6,4.90,20.00,0.4340,1.40,19.90,22.1,18.10,9.92,...,18.60,4.350,1.350,1.710,0.5490,19.10,13.20,1.070,7.15,Ulothrix
30523,5.19,22.6,2.07,19.40,0.0788,1.67,5.09,22.8,19.20,16.20,...,20.00,1.460,0.411,0.458,0.0597,19.80,17.20,0.197,7.15,Ulothrix
30524,9.21,22.7,2.07,21.00,0.0790,5.81,12.70,22.8,20.10,11.40,...,20.10,1.060,0.257,0.319,0.0341,20.70,11.60,0.198,7.15,Ulothrix
30525,8.21,22.6,1.87,20.50,0.0641,5.96,11.30,22.8,20.20,20.20,...,20.20,1.010,0.285,0.350,0.0314,20.80,20.40,0.162,7.15,Ulothrix


In [4]:
MData['microorganisms'].unique()

array(['Spirogyra', 'Volvox', 'Pithophora', 'Yeast', 'Raizopus',
       'Penicillum', 'Aspergillus sp', 'Protozoa', 'Diatom', 'Ulothrix'],
      dtype=object)

In [5]:
dictForClasses = {k:v for k, v in zip(['Spirogyra', 'Volvox', 'Pithophora', 'Yeast', 'Raizopus',
       'Penicillum', 'Aspergillus sp', 'Protozoa', 'Diatom', 'Ulothrix'], list(range(10)))}

In [6]:
for i in range(30527):
    MData.iloc[i, -1] = dictForClasses[MData.iloc[i, -1]]

In [7]:
MData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30527 entries, 0 to 30526
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Solidity         30527 non-null  float64
 1   Eccentricity     30527 non-null  float64
 2   EquivDiameter    30527 non-null  float64
 3   Extrema          30527 non-null  float64
 4   FilledArea       30527 non-null  float64
 5   Extent           30527 non-null  float64
 6   Orientation      30527 non-null  float64
 7   EulerNumber      30527 non-null  float64
 8   BoundingBox1     30527 non-null  float64
 9   BoundingBox2     30527 non-null  float64
 10  BoundingBox3     30527 non-null  float64
 11  BoundingBox4     30527 non-null  float64
 12  ConvexHull1      30527 non-null  float64
 13  ConvexHull2      30527 non-null  float64
 14  ConvexHull3      30527 non-null  float64
 15  ConvexHull4      30527 non-null  float64
 16  MajorAxisLength  30527 non-null  float64
 17  MinorAxisLen

In [8]:
modelForPCA = PCA(n_components=10).fit_transform(MData.iloc[:, 0:24])

In [9]:
X = np.array(modelForPCA)
y = np.asarray(MData.iloc[:, 24]).astype('int64')

In [10]:
X

array([[-2.09745388e+01,  1.59603239e+00, -9.91053784e+00, ...,
        -2.87740455e-01, -7.08252964e-01,  3.67070098e-01],
       [-1.45658752e+01,  1.19409108e+01,  8.73514558e+00, ...,
        -1.20892009e+00,  1.71041494e-02,  3.51093625e-01],
       [-1.37422741e+01, -5.57507471e-01,  8.24498800e+00, ...,
        -8.32251960e-01, -2.20291635e-01,  9.72167910e-01],
       ...,
       [ 2.37173161e+01,  2.07359440e-01,  2.06157881e+00, ...,
         1.53960717e+00,  3.46630861e-01,  9.57946310e-01],
       [ 2.39974267e+01,  1.25150374e+01,  2.35956808e+00, ...,
         1.31225190e+00,  3.81555901e-01,  1.71767737e+00],
       [ 2.51675860e+01,  9.13631241e+00,  7.47669893e+00, ...,
         5.79546863e-01,  2.29249182e-01,  6.42199981e-01]])

In [11]:
y

array([0, 0, 0, ..., 9, 9, 9])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = AutoML()

model.fit(X_train, y_train, task='classification', time_budget=10, metric='accuracy')

[flaml.automl: 05-17 07:19:19] {2105} INFO - task = classification
[flaml.automl: 05-17 07:19:19] {2107} INFO - Data split method: stratified
[flaml.automl: 05-17 07:19:19] {2111} INFO - Evaluation method: holdout
[flaml.automl: 05-17 07:19:19] {2188} INFO - Minimizing error metric: 1-accuracy
[flaml.automl: 05-17 07:19:19] {2281} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 05-17 07:19:19] {2567} INFO - iteration 0, current learner lgbm
[flaml.automl: 05-17 07:19:19] {2698} INFO - Estimated sufficient time budget=3432s. Estimated necessary time budget=84s.
[flaml.automl: 05-17 07:19:19] {2750} INFO -  at 0.8s,	estimator lgbm's best error=0.6177,	best estimator lgbm's best error=0.6177
[flaml.automl: 05-17 07:19:19] {2567} INFO - iteration 1, current learner lgbm
[flaml.automl: 05-17 07:19:19] {2750} INFO -  at 0.9s,	estimator lgbm's best error=0.6177,	best estimator lgbm's best error=0.6177
[flam

In [13]:
print('Best ML Model:', model.best_estimator)
print('Best hyperparmeter config:', model.best_config)
print('Best Accuracy on validation data: %f'%(1 - model.best_loss))
print('Training duration of best run: %f s'%(model.best_config_train_time))

Best ML Model: lgbm
Best hyperparmeter config: {'n_estimators': 154, 'num_leaves': 18, 'min_child_samples': 3, 'learning_rate': 0.17402065726724145, 'log_max_bin': 8, 'colsample_bytree': 0.6649148062238498, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.006761362450996487}
Best Accuracy on validation data: 0.979556
Training duration of best run: 3.785859 s


In [14]:
y_pred = model.predict(X_test)

In [15]:
print(classification_report(y_pred, y_test, target_names=dictForClasses.keys()))

                precision    recall  f1-score   support

     Spirogyra       0.83      1.00      0.91       120
        Volvox       1.00      0.97      0.99      1080
    Pithophora       0.94      0.99      0.96       328
         Yeast       1.00      0.98      0.99       920
      Raizopus       1.00      1.00      1.00       603
    Penicillum       1.00      1.00      1.00       280
Aspergillus sp       0.94      0.98      0.96       927
      Protozoa       1.00      0.99      1.00      1010
        Diatom       1.00      0.99      1.00       453
      Ulothrix       0.98      0.97      0.98      1911

      accuracy                           0.98      7632
     macro avg       0.97      0.99      0.98      7632
  weighted avg       0.98      0.98      0.98      7632

